In [1]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt

import sys
import os

cwd = os.getcwd()

basedir = os.path.abspath(os.path.join(cwd, os.pardir))
if basedir not in sys.path:
    sys.path.insert(0, basedir)


plt.rcParams.update({
    'figure.figsize': (16, 4),
    'font.size': 16,
    'lines.linewidth': 1.5
})

datadir = os.path.join(basedir, 'vessel_data/CSAD')
files = os.listdir(datadir)
filepaths = [os.path.join(datadir, f) for f in files if f.endswith('.mat')]

# Note that the vesselABC file does not seem to be needed. Don't know what it has been used for. 


In [2]:
data = {files[i].split('.mat')[0]: loadmat(filepaths[i]) for i in range(len(filepaths))}

In [3]:
QTF_diag_surge = np.asarray(data['CSAD_model']['vessel']['driftfrc'][0, 0]['amp'][0, 0][0, 0][:, :, 0])
QTF_diag_sway = np.asarray(data['CSAD_model']['vessel']['driftfrc'][0, 0]['amp'][0, 0][0, 1][:, :, 0])
QTF_diag_heave = np.asarray(data['CSAD_model']['vessel']['driftfrc'][0, 0]['amp'][0, 0][0, 2][:, :, 0])
QTF_diag_roll = np.asarray(data['CSAD_model']['vessel']['driftfrc'][0, 0]['amp'][0, 0][0, 3][:, :, 0])
QTF_diag_pitch = np.asarray(data['CSAD_model']['vessel']['driftfrc'][0, 0]['amp'][0, 0][0, 4][:, :, 0])
QTF_diag_yaw = np.asarray(data['CSAD_model']['vessel']['driftfrc'][0, 0]['amp'][0, 0][0, 5][:, :, 0])

In [4]:
QTF_diag_surge_beta0 = QTF_diag_surge[:, 0].copy()
QTF_diag_surge_beta36 = QTF_diag_surge[:, 35].copy()

In [5]:
data['CSAD_model']['vessel']['driftfrc'][0,0]['amp'][0,0][0, 1][:, :, 0].shape

(47, 36)

In [6]:
freqs = np.asarray(data['CSAD_model']['vessel']['driftfrc'][0, 0]['w'][0,0][0, :])
headings = np.asarray(data['CSAD_model']['vessel']['headings'][0, 0][0, :])

In [7]:
# Test out algorithm used by Brørby for selecting angle. 
np.degrees(headings[np.argmin(np.abs(headings - np.deg2rad(15.)))])

10.0

In [8]:
from waves.wave_loads import WaveLoad

In [9]:
wave_freqs = np.linspace(0.5, 7, 50)

waveload = WaveLoad(np.linspace(0, 2.5, len(wave_freqs)), wave_freqs, np.random.uniform(0, 6.28, size=len(freqs)), headings, 0)

qtf_diag_surge = np.asarray(data['CSAD_model']['vessel']['driftfrc'][0, 0]['amp'][0, 0][0, 0][:, :, :])

In [10]:
full_qtf_surge_heading0 = waveload._full_qtf(headings, freqs, qtf_diag_surge, 0, method="")


Execution time of _full_qtf: 0.0189


In [11]:
print(full_qtf_surge_heading0)

[[1.74824926e-03 2.79718314e-03 4.54245880e-03 ... 2.02929098e+00
  1.32182174e+00 1.32182174e+00]
 [2.79718314e-03 4.47546796e-03 7.26789337e-03 ... 3.24684739e+00
  2.11490294e+00 2.11490294e+00]
 [4.54245880e-03 7.26789337e-03 1.18026259e-02 ... 5.27268675e+00
  3.43447639e+00 3.43447639e+00]
 ...
 [2.02929098e+00 3.24684739e+00 5.27268675e+00 ... 2.35551188e+03
  1.53431264e+03 1.53431264e+03]
 [1.32182174e+00 2.11490294e+00 3.43447639e+00 ... 1.53431264e+03
  9.99407097e+02 9.99407097e+02]
 [1.32182174e+00 2.11490294e+00 3.43447639e+00 ... 1.53431264e+03
  9.99407097e+02 9.99407097e+02]]


In [12]:
QTF_diag_surge_beta0

array([4.86661285e-05, 1.01960638e-04, 2.52622544e-04, 3.11623371e-04,
       3.89088820e-04, 4.91090980e-04, 6.29965484e-04, 7.17648219e-04,
       8.21388885e-04, 9.44606830e-04, 1.09291433e-03, 1.26851816e-03,
       1.48405153e-03, 1.74824926e-03, 2.07429047e-03, 2.48005235e-03,
       2.98809474e-03, 3.64012335e-03, 4.47546796e-03, 5.59009633e-03,
       7.04724367e-03, 9.04477379e-03, 1.18026259e-02, 1.57227635e-02,
       2.14435362e-02, 3.01072378e-02, 4.37648608e-02, 6.64250444e-02,
       1.06324207e-01, 1.82583235e-01, 3.45914483e-01, 7.55869233e-01,
       2.06969239e+00, 2.63168924e+00, 4.12212272e+00, 6.84665024e+00,
       1.20650551e+01, 2.28807072e+01, 4.64502815e+01, 1.01643078e+02,
       2.34535457e+02, 5.54521501e+02, 1.23450924e+03, 2.06632456e+03,
       2.35551188e+03, 9.99407097e+02, 1.78730223e+02])

In [13]:
m = np.array([
    [1, 0],
    [0, 0]
])

t = 0

m = m*t

np.exp(m)

array([[1., 1.],
       [1., 1.]])

In [14]:
# Antall bølge frekvenser
a1, a2 = 1, 2
w1, w2 = 1, 0.5
p1, p2 = 0, 1

np.random.seed(25)
N = 100 # Number of wave components
w = np.random.random(size=N*N)
a = np.random.random(size=N)
p = np.random.random(size=N*N)

# Q = np.array([
#     [0.5, 0.75],
#     [0.75, 1]
# ])
# Random QTF
Q = np.random.random(size=N*N).reshape(N, N)

A = a.copy()

W = np.zeros_like(Q)
P = np.zeros_like(Q)
for i in range(N):
    for j in range(N):
        W[i, j] = w[i] - w[j]
        P[i, j] = p[i] - p[j]



t = 0
from time import time
t1 = time()
res = np.real(A.T@(Q*np.exp(W*(1j*t) + P))@A)
t2 = time()
print(f"Result: {res}")
print(f"Execution time {1000*(t2 - t1)} [ms]")

Result: 1327.2243838296745
Execution time 5.985736846923828 [ms]


In [15]:
res = 0
t1 = time()
for i in range(N):
    for j in range(N):
        Qij = Q[i, j]
        res += a[i]*a[j]*Qij*np.exp(1j*(w[i]-w[j])*t + (p[i] - p[j]))

t2 = time()
print(f"Result: {np.real(res)}")
print(f"Execution time: {1000*(t2 - t1):.2f} [ms]")



Result: 1327.2243838296747
Execution time: 54.85 [ms]


In [16]:
np.cos([
    [np.pi/2, 0],
    [0, np.pi/2]
])

array([[6.123234e-17, 1.000000e+00],
       [1.000000e+00, 6.123234e-17]])